In [1]:
import numpy as np

import adaptoctree.morton as morton
import adaptoctree.tree as tree
import adaptoctree.plotting as plotting

In [2]:
N = int(1000)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 50


max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

In [3]:
unbalanced = tree.build(particles, max_level, max_num_particles)
unbalanced = np.unique(unbalanced)
depth = max(morton.find_level(unbalanced))
unbalanced

array([       1,    65537,   163841,   229377,  3735555,  3768323,
        3866627,  3899395,  3997699,  4128771,  4161539,  7340035,
        7372803,  7471107,  7503875,  7602179,  7733251,  7766019,
        8978435,  9011203,  9109507,  9142275,  9240579,  9273347,
        9371651, 12582915, 12615683, 12713987, 12746755, 12845059,
       12877827, 12976131])

In [4]:
balanced = tree.balance(unbalanced, depth)
len(balanced)
# balanced = np.array(balanced)
# balanced.sort()
# balanced

206

In [5]:
def assign_points_to_keys(points, tree, x0, r0):
    """
    Assign particle positions to Morton keys in a given tree.
    Parameters:
    -----------
    points : np.array(shape=(N, 3), dtype=np.float32)
    tree : Octree
    Returns:
    --------
    np.array(shape=(N,), dtype=np.int64)
        Column vector specifying the Morton key of the node that each point is
        associated with.
    """
    # Map Morton key to bounds that they represent.
    n_points = points.shape[0]
    n_keys = len(tree)
    leaves = np.zeros(n_points, dtype=np.int64)

    # Loop over points, and assign to a node from the tree by examining the bounds
    for i, point in enumerate(points):
        for key in tree:
            lower_bound, upper_bound = morton.find_node_bounds(key, x0, r0)
            if (np.all(lower_bound <= point)) and (np.all(point < upper_bound )):
                leaves[i] = key        

    return leaves

In [6]:
balanced_leaves = assign_points_to_keys(particles, balanced, x0, r0)

In [7]:
# Test num particles constraint

_, counts = np.unique(balanced_leaves, return_counts=True)
assert np.all(counts) < max_num_particles

In [8]:
def are_neighbours(a, b, x0, r0):
    
    if a in morton.find_ancestors(b):
        return False
    if b in morton.find_ancestors(a):
        return False
    
    level_a = morton.find_level(a)
    radius_a = r0 / (1 << level_a)
    
    level_b = morton.find_level(b)
    radius_b = r0 / (1 << level_b)
    
    centre_a = morton.find_center_from_key(a, x0, r0)
    centre_b = morton.find_center_from_key(b, x0, r0)
    
    if np.linalg.norm(centre_a - centre_b) <= np.sqrt(3) * (radius_b + radius_a):
        return True
    return False

In [9]:
# Check for balancing condition

for i in balanced:
    for j in balanced:
        if (i != j) and are_neighbours(i, j, x0, r0):
            diff = morton.find_level(i) - morton.find_level(j)
            assert diff <= 1

In [10]:
# Check for overlaps

for i, ki in enumerate(balanced):
    for j, kj in enumerate(balanced):
        if j != i:
            assert ki not in morton.find_ancestors(kj)            